# Wrist Movement Data Formating

In [1]:
import numpy as np
import pandas as pd
from math import sqrt,acos,degrees
import matplotlib.pyplot as plt
from scipy.signal import resample
from tqdm.notebook import tqdm
from time import time
import os
import csv

## EMG Data Extraction (Cumilative Function)

In [2]:
def extract_dataframes(path,file,save=False):
    file_s = file
    file = '/marker_data/' + file
    
    with open(path+'/'+file, "r") as f:
        lines = f.readlines()
    emg_labels = ['Frame','Sub Frame',
                 'IM EMG1',
                 'IM EMG2',
                 'IM EMG3',
                 'IM EMG4',
                 'IM EMG5',
                 'IM EMG6',
                 'IM EMG7',
                 'IM EMG8',
                 'IM EMG9',
                 'IM EMG10',
                 'IM EMG11',
                 'IM EMG12']
    emg_labels_ref = ['Frame','Sub Frame',
                 'EMG1',
                 'EMG2',
                 'EMG3',
                 'EMG4',
                 'EMG5',
                 'EMG6',
                 'EMG7',
                 'EMG8',
                 'EMG9',
                 'EMG10',
                 'EMG11',
                 'EMG12']
    marker_labels = ['Frame','Sub Frame',
                  'RSHO_X','RSHO_Y','RSHO_Z',
                  'RUPA_X','RUPA_Y','RUPA_Z',
                  'RELB_X','RELB_Y','RELB_Z',
                  'RM1_X','RM1_Y','RM1_Z',
                 'RFRM_X','RFRM_Y','RFRM_Z',
                 'WRM2_X','WRM2_Y','WRM2_Z',
                 'RWRA_X','RWRA_Y','RWRA_Z',
                 'RWRB_X','RWRB_Y','RWRB_Z',
                 'RFIN_X','RFIN_Y','RFIN_Z']
    
    pronation_movement = ['No Motion','Supination','Pronation']
    flexion_movement = ['No Motion','Extension','Flexion']
    radial_movement = ['No Motion','Ulnar','Radial']
    dtm_movement = ['No Motion','Forward','Backward']
    
    #################
    # EMG Data Frame#
    #################
    
    emg_lines = []
    for line in lines[5:]:
        if line=='\n':
            break
        emg_lines.append(line.split(','))
    emg_df = pd.DataFrame(np.array(emg_lines),columns=lines[3].split(','))
    emg_df = emg_df[emg_labels]
    emg_df.columns = emg_labels_ref
    emg_df = emg_df[emg_df.columns].astype(float)
    duration = emg_df.shape[0]/2000
    
    # Marker Data Frame
    marker_lines = []
    marker_line_start = None
    for i in range(len(lines)):
        if lines[i]=='Trajectories\n':
            marker_line_start = i
            break
    for line in lines[marker_line_start+5:]:
        if line=='\n':
            break
        marker_lines.append(line.split(','))
    marker_df = pd.DataFrame(np.array(marker_lines),columns=marker_labels)
    marker_df = marker_df[marker_df.columns].astype(float)
    
    # Angles Dataframe
    angles_df = compute_wrist_angles(marker_df,degree=True)
    
    pronations = np.array(angles_df['Pronation_Angle'])
    flexions = np.array(angles_df['Flexion_Angle'])
    radials = np.array(angles_df['Radial_Angle'])
    elbows = np.array(angles_df['Elbow_Joint'])
    
    # Resampling to EMG SR(2000 Hz) from Vicon SR(100 Hz)
    pronations = resample_series(pronations,100,2000)
    flexions = resample_series(flexions,100,2000)
    radials = resample_series(radials,100,2000)
    elbows = resample_series(elbows,100,2000)    
    
    diff_interval = 1000
    
    pronation_labels,pronation_movement_labels = direction_labels(pronations,diff_interval,pronation_movement)
    flexion_labels,flexion_movement_labels = direction_labels(flexions,diff_interval,flexion_movement)
    radial_labels,radial_movement_labels = direction_labels(radials,diff_interval,radial_movement)
    
    # Dart Throwing Motion Labeling
    dtm_labels = []
    ln = len(flexion_labels)
    
    for i in range(ln):
        if(flexion_labels[i]==0 and radial_labels[i]==0):
            dtm_labels.append(0)
        elif(flexion_labels[i]==2 and radial_labels[i]==1):
            dtm_labels.append(1)
        elif(flexion_labels[i]==1 and radial_labels[i]==2):
            dtm_labels.append(2)
        else:
            dtm_labels.append(0)
    
    emg_df['Pronation_Angle'] = pronations
    emg_df['Pronation_Label'] = pronation_labels
    
    emg_df['Flexion_Angle'] = flexions
    emg_df['Flexion_Label'] = flexion_labels
    
    emg_df['Radial_Angle'] = radials
    emg_df['Radial_Label'] = radial_labels
    
    emg_df['Elbow_Joint_Angle'] = elbows
    
    emg_df['DTM_Label'] = dtm_labels
    
    if(save==True):
        emg_df.to_csv(path+'/computed_'+file_s)
        
    return emg_df,marker_df, angles_df

In [ ]:
def resample_series(data,sr_origin,sr_new):
    """
    Upsamples Series Vector to required Freq(Hz)
    data - Series 1D Array
    sr_origin - Origin Sampling Rate
    sr_new - New Sampling Rate
    Return - Resampled Data to Given Sample Rate
    """
    dt = pd.Series(data)
    dt.index = pd.to_datetime(dt.index*(int((sr_origin/10)**7)))
    dt2 = dt.resample(str(1/sr_new)+'S').ffill()
    len_diff = len(dt)*(sr_new/sr_origin) - len(dt2)
    dt2_list = list(np.array(dt2))
    resampled_array = None
    
    # Balencing
    if(len_diff%2==0):
        nd = int(len_diff/2)
        resampled_array = [dt2_list[0]]*nd + dt2_list
        resampled_array = resampled_array + [dt2_list[-1]]*nd
    else:
        nd = int((len_diff+1)/2)
        resampled_array = [dt2_list[0]]*nd + dt2_list
        resampled_array = resampled_array + [dt2_list[-1]]*nd
        resampled_array = resampled_array[1:]
        
    return np.array(resampled_array)

In [3]:
def resample_series(data,sr_origin,sr_new):
    """
    Upsamples Series Vector to required Freq(Hz)
    data - Series 1D Array
    sr_origin - Origin Sampling Rate
    sr_new - New Sampling Rate
    Return - Resampled Data to Given Sample Rate
    """
    data = np.array(data)
    ln = data.shape[0]
    new_ln = int(ln*(sr_new/sr_origin))
    resampled_array = resample(data,new_ln)    
    return np.array(resampled_array)

## Compute Wrist Angles

In [4]:
def compute_wrist_angles(df,degree=False):
    # Wrist Segment
    WRM2 = df[['WRM2_X','WRM2_Y','WRM2_Z']]
    RWRA = df[['RWRA_X','RWRA_Y','RWRA_Z']]
    RWRB = df[['RWRB_X','RWRB_Y','RWRB_Z']]
    # Palm Segment
    RFIN = df[['RFIN_X','RFIN_Y','RFIN_Z']]

    # Elbow Segment
    RFRM = df[['RFRM_X','RFRM_Y','RFRM_Z']]
    RM1 = df[['RM1_X','RM1_Y','RM1_Z']]
    RELB = df[['RELB_X','RELB_Y','RELB_Z']]
    # Shoulder Segment
    RSHO = df[['RSHO_X','RSHO_Y','RSHO_Z']]
    RUPA = df[['RUPA_X','RUPA_Y','RUPA_Z']]
    # Bisector Point
    MID = (np.array(RWRB) + np.array(RWRA))/2
    MIDE = (np.array(RFRM) + np.array(RM1))/2

    # Translate Wrist to Elbow Segment Mid
    RWRB_E = RWRB - MIDE

    flexion_angles = angles_lines(RFIN,WRM2,MID,deg=degree)
    radial_angles = angles_lines(RFIN,RWRB,MID,deg=degree)-90
    pronation_angles = angles_lines(RFRM,RWRB_E,MIDE,deg=degree)
    elbow_angles = angles_lines(RSHO,MID,MIDE,deg=degree)
    
    df_labels = ['Flexion_Angle','Radial_Angle','Pronation_Angle','Elbow_Joint']
#     df_labels = ['Pitch','Yaw','Roll','Elbow_Joint']
    ndf = pd.DataFrame(columns=df_labels)
    ndf['Flexion_Angle'] = flexion_angles
    ndf['Radial_Angle'] = radial_angles
    ndf['Pronation_Angle'] = pronation_angles
    ndf['Elbow_Joint'] = elbow_angles
    return ndf

def angles_lines(p1,p2,mid,deg=False):
    u = np.array(p1)-np.array(mid)
    v = np.array(p2)-np.array(mid)
    i1,j1,k1 = u[:,0],u[:,1],u[:,2]
    i2,j2,k2 = v[:,0],v[:,1],v[:,2]
    angles = []
    for t in range(len(i1)):
        cos_t = abs(i1[t]*i2[t]+j1[t]*j2[t]+k1[t]*k2[t])
        cos_t = cos_t/(sqrt(i1[t]**2+j1[t]**2+k1[t]**2)*sqrt(i2[t]**2+j2[t]**2+k2[t]**2))
        if deg==False:
            angles.append(acos(cos_t))  
        if deg==True:
            angles.append(degrees(acos(cos_t)))
    return np.array(angles)

## Movement Labelling

In [5]:
def direction_labels(array,interval=50,movements=None,angle_thresh=5):
    """
    0 - No Motion
    1 - Positive Direction
    2 - Negative Direction
    """
    labels = []
    diff_arr = difference(array,interval)
    data_len = len(array)
    len_diff = int(data_len - len(diff_arr))
    
    # Hot Labelling
    for diff in diff_arr:
        if(abs(diff)>angle_thresh):
            labels.append(0)
        elif(diff>0):
            labels.append(1)
        elif(diff<0):
            labels.append(2)
            
    # Balencing
    if(len_diff%2==0):
        nd = int(len_diff/2)
        labels = [labels[0]]*nd + labels
        labels = labels + [labels[-1]]*nd
    else:
        nd = int((len_diff+1)/2)
        labels = [labels[0]]*nd + labels
        labels = labels + [labels[-1]]*nd 
        labels = labels[1:]
        
    # Movement Labelling
    if(movements==None):
        return np.array(labels)
    else:
        movement_labels = []
        hot_labels = labels
        for lb in labels:
            movement_labels.append(movements[lb])
        return hot_labels,movement_labels

In [6]:
def direction_labels(array,interval=50,movements=None,angle_thresh=3):
    """
    0 - No Motion
    1 - Positive Direction
    2 - Negative Direction
    """
    labels = []
    data = np.array(array)
    avg = (data.max()-data.min())/2
    for x in list(data):
        if(abs(avg-x)<angle_thresh):
            labels.append(0)
        else:
            labels.append(3)
            
    n = len(array)/2000
    split_list = split(data,int(n))
    group_labels = []
    for arr in split_list:
        if(difference(arr).mean()>0):
            group_labels.append(1)
        else:
            group_labels.append(2)
    direction_labels = []        
    for gl in group_labels:
        for i in range(2000):
            direction_labels.append(gl)
    for i in range(len(direction_labels)):
        if(labels[i]!=0):
            labels[i]=direction_labels[i]
            
    # Movement Labelling
    if(movements==None):
        return np.array(labels)
    else:
        movement_labels = []
        hot_labels = labels
        for lb in labels:
            movement_labels.append(movements[lb])
        return hot_labels,movement_labels

def split(a, n):
    k, m = divmod(len(a), n)
    return list((a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)))

def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return np.array(diff)


In [7]:
def direction_labels(array,interval=50,movements=None):
    """
    0 - No Motion
    1 - Positive Direction
    2 - Negative Direction
    """
    labels = [0]*interval
    i=interval
    std = np.array(array).std()
    while(len(labels)<len(array)):
        diff = difference(array[i-interval:i]).mean()
        if(abs(diff)<std/1000):
            labels.append(0)
        elif(diff>0):
            labels.append(1)
        elif(diff<0):
            labels.append(2)
        i=i+1
    # Movement Labelling
    if(movements==None):
        return np.array(labels)
    else:
        movement_labels = []
        hot_labels = labels
        for lb in labels:
            movement_labels.append(movements[lb])
        return hot_labels,movement_labels

def difference(dataset, interval=1):
	diff = []
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return np.array(diff)


## Data Generation Section

In [8]:
files = []
# trial_names = ['Bulb','Hammer','Cup','Screw']
trial_names = ['Bulb','Hammer','Cup']
for exp in trial_names:
    for i in range(3):
        if(i==0):
            files.append(exp+'.csv')
        else:
            files.append(exp+'0'+str(i)+'.csv')

path = 'Subjects/Fazil/day_04_14_21'
files

['Bulb.csv',
 'Bulb01.csv',
 'Bulb02.csv',
 'Hammer.csv',
 'Hammer01.csv',
 'Hammer02.csv',
 'Cup.csv',
 'Cup01.csv',
 'Cup02.csv']

In [9]:
start = time()
for file in tqdm(files):
    extract_dataframes(path,file,save=True)
    print('Eapsed-',time()-start,'s')
    start = time()

Eapsed- 133.4657952785492 s
Eapsed- 136.19984459877014 s
Eapsed- 133.24952721595764 s
Eapsed- 81.38480806350708 s
Eapsed- 86.56690907478333 s
Eapsed- 83.6885769367218 s
Eapsed- 96.67063474655151 s
Eapsed- 99.8154706954956 s
Eapsed- 96.1421492099762 s



## Testing Section

In [9]:
file = 'Hammer.csv'
emg_df,marker_df, angles_df = extract_dataframes(path,file,save=False)

In [13]:
array = np.array(emg_df['Flexion_Angle'])
labels = direction_labels(array)

In [10]:
emg_df[['Pronation_Angle','Flexion_Angle','Radial_Angle','Elbow_Joint_Angle']].describe()

,Pronation_Angle,Flexion_Angle,Radial_Angle,Elbow_Joint_Angle
count,54000.000000,54000.000000,54000.000000,54000.000000
mean,74.726219,53.608026,-2.860676,34.656572
std,15.863335,16.448584,3.189437,5.335869
min,15.028426,23.145036,-10.840965,23.723008
25%,78.627578,40.278069,-5.900567,30.956145
50%,80.697719,55.321790,-0.911475,36.129522
75%,81.772450,70.280929,-0.310285,39.210870
max,89.993319,77.486824,-0.000027,44.027489


In [140]:
emg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54000 entries, 0 to 53999
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Frame              54000 non-null  float64
 1   Sub Frame          54000 non-null  float64
 2   EMG1               54000 non-null  float64
 3   EMG2               54000 non-null  float64
 4   EMG3               54000 non-null  float64
 5   EMG4               54000 non-null  float64
 6   EMG5               54000 non-null  float64
 7   EMG6               54000 non-null  float64
 8   EMG7               54000 non-null  float64
 9   EMG8               54000 non-null  float64
 10  EMG9               54000 non-null  float64
 11  EMG10              54000 non-null  float64
 12  EMG11              54000 non-null  float64
 13  EMG12              54000 non-null  float64
 14  Pronation_Angle    54000 non-null  float64
 15  Pronation_Label    54000 non-null  int64  
 16  Flexion_Angle      540

In [141]:
marker_df

,Frame,Sub Frame,RSHO_X,RSHO_Y,RSHO_Z,RUPA_X,RUPA_Y,RUPA_Z,RELB_X,RELB_Y,...,WRM2_Z,RWRA_X,RWRA_Y,RWRA_Z,RWRB_X,RWRB_Y,RWRB_Z,RFIN_X,RFIN_Y,RFIN_Z
0,1.0,0.0,0.786794,1376.67,1616.09,-135.4000,1412.46,1607.73,-285.682,1444.14,...,1648.15,-603.212,1365.72,1645.05,-598.752,1426.23,1634.32,-693.247,1392.37,1661.16
1,2.0,0.0,0.809722,1376.66,1616.10,-135.3830,1412.41,1607.74,-285.677,1444.08,...,1648.22,-603.181,1365.57,1645.12,-598.727,1426.06,1634.36,-693.230,1392.17,1661.19
2,3.0,0.0,0.846024,1376.67,1616.12,-135.3870,1412.40,1607.75,-285.665,1444.00,...,1648.27,-603.146,1365.40,1645.18,-598.727,1425.91,1634.43,-693.226,1391.99,1661.25
3,4.0,0.0,0.878274,1376.64,1616.14,-135.3790,1412.34,1607.76,-285.661,1443.91,...,1648.32,-603.130,1365.22,1645.21,-598.733,1425.73,1634.48,-693.202,1391.78,1661.27
4,5.0,0.0,0.909995,1376.65,1616.16,-135.3500,1412.28,1607.81,-285.650,1443.81,...,1648.36,-603.088,1365.05,1645.28,-598.689,1425.55,1634.51,-693.178,1391.59,1661.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,2696.0,0.0,43.278200,1350.12,1620.58,-84.3952,1388.82,1632.85,-232.038,1439.42,...,1685.84,-555.845,1372.15,1680.27,-546.822,1432.62,1679.48,-644.942,1405.98,1699.16
2696,2697.0,0.0,43.225100,1349.94,1620.59,-84.4793,1388.66,1632.85,-232.100,1439.23,...,1685.88,-555.933,1372.06,1680.32,-546.873,1432.54,1679.47,-645.021,1405.94,1699.21
2697,2698.0,0.0,43.207000,1349.78,1620.54,-84.5045,1388.50,1632.85,-232.179,1439.05,...,1685.94,-556.014,1371.98,1680.40,-546.969,1432.47,1679.48,-645.090,1405.89,1699.27
2698,2699.0,0.0,43.129400,1349.62,1620.60,-84.5657,1388.39,1632.89,-232.204,1438.87,...,1686.02,-556.013,1371.88,1680.50,-546.989,1432.37,1679.57,-645.089,1405.82,1699.31


In [142]:
angles_df

,Flexion_Angle,Radial_Angle,Pronation_Angle,Elbow_Joint
0,26.986938,-8.467178,41.598246,25.664406
1,26.993573,-8.490643,41.567447,25.681117
2,26.962000,-8.465988,41.578738,25.693798
3,26.971479,-8.459310,41.579743,25.696435
4,26.959687,-8.480352,41.609291,25.688764
...,...,...,...,...
2695,23.148273,-6.307393,23.922245,23.822728
2696,23.185359,-6.324047,23.923770,23.837621
2697,23.207613,-6.306773,24.011918,23.811029
2698,23.145036,-6.271629,24.117053,23.813491
